# CP2 - Relatório com Análise dos Dados Automatizada

## Integrantes
- **Rony Ken Nagai** - RM: 551549
- **Tomáz Versolato Carballo** - RM: 551417

## Objetivo do Trabalho
O objetivo deste trabalho é desenvolver um sistema automatizado para análise de dados utilizando Python e bibliotecas populares como `pandas`, `matplotlib` e `seaborn`, além de empregar o `jinja2` para geração de relatórios HTML estilizados com Bootstrap. O sistema recebe um dataset e gera automaticamente um relatório com as principais estatísticas e gráficos, facilitando a análise exploratória.

## Estrutura do Código

### Bibliotecas Utilizadas
- **pandas**: Manipulação e análise dos dados em formato de dataframes.
- **matplotlib e seaborn**: Geração de gráficos para visualização dos dados.
- **jinja2**: Motor de templates para renderizar o HTML do relatório.
- **os, base64 e BytesIO**: Manipulação de arquivos e conversão de gráficos para Base64 para inclusão no HTML.

### Passo a Passo do Código

1. **Configuração do Template HTML:**
   - O código cria um template HTML utilizando o `Bootstrap` para estilização. O template inclui cards para mostrar as estatísticas e gráficos gerados, garantindo uma apresentação clara e organizada das informações.

2. **Função `resumo_df(df)`:**
   - Calcula as estatísticas principais do dataset, como:
     - Número de variáveis.
     - Número de observações.
     - Células faltantes.
     - Linhas duplicadas.
     - Quantidade de variáveis numéricas e categóricas.
   - Retorna essas informações em um dicionário chamado `summary`.

3. **Função `plot_64(fig)`:**
   - Converte os gráficos gerados em uma string Base64 para que possam ser incorporados no HTML diretamente. Isso garante que os gráficos sejam exibidos corretamente independentemente do ambiente.

4. **Função `criar_plot(df)`:**
   - Gera gráficos para variáveis numéricas (histogramas) e categóricas (gráficos de contagem).
   - Utiliza o `seaborn` para estilizar os gráficos e `matplotlib` para renderizá-los.

5. **Função `gerar_html(df)`:**
   - Renderiza o template HTML utilizando os dados gerados pelas funções anteriores.
   - Insere as estatísticas e gráficos no relatório e salva o arquivo HTML no ambiente do Colab.

6. **Execução Principal (`if __name__ == '__main__'`):**
   - Carrega o dataset `Diabetes.csv` e gera o relatório HTML utilizando as funções definidas.
   - Exibe o relatório diretamente no notebook do Colab utilizando `IFrame` para visualização.

## Estrutura do Relatório

O relatório HTML gerado possui as seguintes seções:

- **Título e Sumário:** Exibe informações básicas como o número de linhas, colunas, variáveis, observações, células faltantes e linhas duplicadas.
- **Tipos de Dados:** Lista os tipos de dados presentes em cada coluna.
- **Tipos de Variáveis:** Informa a quantidade de variáveis numéricas e categóricas.
- **Valores Faltantes:** Mostra a quantidade de valores faltantes em cada coluna.
- **Gráficos:** Exibe gráficos exploratórios para variáveis numéricas e categóricas, organizados em cards responsivos.

## Como Executar
1. Certifique-se de que todas as bibliotecas necessárias estão instaladas no ambiente Colab.
2. Execute o código e carregue o dataset desejado. Ao utilizar outro dataset, certifique-se de alterar no código o nome do dataset: `df = pd.read_csv('/content/nome-do-dataset.csv')`
3. O relatório será gerado e exibido automaticamente na interface do Colab.

## Conclusão
Este projeto demonstra como é possível automatizar a análise exploratória de dados de forma eficiente e visualmente atrativa. O uso de `pandas` para manipulação dos dados, aliado às bibliotecas de visualização e ao uso de templates HTML com `jinja2`, permite a criação de relatórios ricos em informações e acessíveis.

---

In [27]:
# Instalação de bibliotecas necessárias
!pip install jinja2
!pip install matplotlib seaborn pandas


In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from jinja2 import Environment, FileSystemLoader
import os
import base64
from io import BytesIO

# Configuração do ambiente para Jinja2 no Colab
!mkdir -p /content/templates
template_html = """
<!DOCTYPE html>
<html lang="pt-br">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Relatório com Análise dos Dados Automatizada</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">
    <div class="container my-5">
        <h1 class="text-center mb-4">Relatório com Análise dos Dados Automatizada</h1>
        <div class="card mb-4">
            <div class="card-body">
                <h2 class="card-title">Sumário</h2>
                <p><strong>Número de Linhas:</strong> {{ summary.shape[0] }}</p>
                <p><strong>Número de Colunas:</strong> {{ summary.shape[1] }}</p>
                <p><strong>Número de Variáveis:</strong> {{ summary.num_var }}</p>
                <p><strong>Número de Observações:</strong> {{ summary.num_obs }}</p>
                <p><strong>Células Faltantes:</strong> {{ summary.cels_faltantes }}</p>
                <p><strong>Linhas Duplicadas:</strong> {{ summary.linhas_duplicadas }}</p>
            </div>
        </div>

        <div class="card mb-4">
            <div class="card-body">
                <h3 class="card-title">Tipos de Data</h3>
                <ul class="list-group">
                    {% for column, dtype in summary.dtypes.items() %}
                    <li class="list-group-item">{{ column }}: {{ dtype }}</li>
                    {% endfor %}
                </ul>
            </div>
        </div>

        <div class="card mb-4">
            <div class="card-body">
                <h3 class="card-title">Tipos de Variáveis</h3>
                <p><strong>Numérico:</strong> {{ summary.numeric_vars }}</p>
                <p><strong>Categórico:</strong> {{ summary.categorical_vars }}</p>
            </div>
        </div>

        <div class="card mb-4">
            <div class="card-body">
                <h3 class="card-title">Valores Faltantes</h3>
                <ul class="list-group">
                    {% for column, missing in summary.valores_faltantes.items() %}
                    <li class="list-group-item">{{ column }}: {{ missing }}</li>
                    {% endfor %}
                </ul>
            </div>
        </div>

        <h2 class="mt-5">Gráficos</h2>
        <div class="row">
            {% for plot in plots %}
            <div class="col-md-6 mb-4">
                <div class="card">
                    <img src="data:image/png;base64,{{ plot }}" class="card-img-top" alt="Plot">
                </div>
            </div>
            {% endfor %}
        </div>
    </div>

    <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.9.2/dist/umd/popper.min.js"></script>
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
</body>
</html>
"""

# Salvando o template HTML no sistema de arquivos do Colab
with open('/content/templates/report_template.html', 'w') as file:
    file.write(template_html)

# Função para gerar resumo do dataframe
def resumo_df(df):
    num_var = len(df.columns)
    num_obs = len(df)
    cels_faltantes = df.isnull().sum().sum()
    linhas_duplicadas = df.duplicated().sum()

    numeric_vars = len(df.select_dtypes(include=np.number).columns)
    categorical_vars = len(df.select_dtypes(include='object').columns)

    summary = {
        'shape': df.shape,
        'num_var': num_var,
        'num_obs': num_obs,
        'cels_faltantes': cels_faltantes,
        'linhas_duplicadas': linhas_duplicadas,
        'numeric_vars': numeric_vars,
        'categorical_vars': categorical_vars,
        'dtypes': df.dtypes.to_dict(),
        'valores_faltantes': df.isnull().sum().to_dict(),
        'duplicados': df.duplicated().sum(),
        'status_descritiveis': df.describe().to_dict()
    }
    return summary

# Função para converter gráficos para Base64
def plot_64(fig):
    buf = BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    return img_base64

# Função para gerar gráficos e retornar como Base64
def criar_plot(df):
    plots = []

    # Histogramas para variáveis numéricas
    for column in df.select_dtypes(include=np.number).columns:
        fig, ax = plt.subplots()
        sns.histplot(df[column], kde=True, ax=ax)
        plt.title(f'Histograma de {column}')
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    # Gráficos de contagem para variáveis categóricas
    for column in df.select_dtypes(include='object').columns:
        fig, ax = plt.subplots()
        sns.countplot(y=df[column], ax=ax)
        plt.title(f'Countplot de {column}')
        img_base64 = plot_64(fig)
        plots.append(img_base64)
        plt.close(fig)

    return plots

# Função para gerar relatório HTML
def gerar_html(df):
    env = Environment(loader=FileSystemLoader('/content/templates'))
    template = env.get_template('report_template.html')

    summary = resumo_df(df)
    plots = criar_plot(df)

    html_content = template.render(summary=summary, plots=plots)

    # Salvar o relatório HTML no Colab
    report_path = '/content/relatorio.html'
    with open(report_path, 'w') as f:
        f.write(html_content)

    print(f'Report saved at: {report_path}')
    return report_path

if __name__ == '__main__':
    df = pd.read_csv('/content/Diabetes.csv')
    report_path = gerar_html(df)

    # Exibir o relatório HTML no Colab
    from IPython.display import IFrame
    IFrame(report_path, width=800, height=600)


Report saved at: /content/relatorio.html
